Let us move further with the coffea output files (we have two files, one for signal and the other for background). We are doing step 2 now.

In [1]:
#Imports:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import sys

sys.path.append("../analysisTools/")
import plotTools as ptools

**Import coffea files -- signal and background**

In [3]:
#coffea output file -- signal
f_sig = "../coffea/skimmed/example_signal_skimmed_SR.coffea"

#Nested Dictionary:
sig_histo = util.load(f_sig) [0]
print (sig_histo.keys())

dict_keys(['bdtScore', 'gen_met_noWgt', 'gen_met', 'gen_leadjet_pt', 'gen_leadjet_pt_noWgt', 'cutDesc', 'cutflow', 'cutflow_cts', 'cutflow_nevts', 'cutflow_vtx_matched'])


We will focus on the last five keys.

In [4]:
print (sig_histo ['cutDesc'])   #represents cut names

defaultdict(<class 'str'>, {'all': 'No cuts', 'hasVtx': 'Baseline Selection', 'cut4': 'computing BDT score', 'cut5': 'No b-tagged jets', 'cut6': 'Leading jet |eta| < 2.4', 'cut7': 'Leading jet pT > 80 GeV', 'cut8': 'dPhi(MET,leading jet) > 1.5', 'cut9': 'dPhi(MET,all jets) > 0.75', 'cut10': 'BDT'})


In [5]:
sig_samples = list(sig_histo['cutflow'].keys()) # names of signal sub-process, i.e. mass point, lifetime etc.
print (sig_samples)

['sig_2018_Mchi-99p0_dMchi-18p0_ctau-100', 'sig_2018_Mchi-94p5_dMchi-9p0_ctau-100', 'sig_2018_Mchi-99p0_dMchi-18p0_ctau-10', 'sig_2018_Mchi-99p0_dMchi-18p0_ctau-1', 'sig_2018_Mchi-88p0_dMchi-16p0_ctau-100', 'sig_2018_Mchi-88p0_dMchi-16p0_ctau-10', 'sig_2018_Mchi-84p0_dMchi-8p0_ctau-100', 'sig_2018_Mchi-94p5_dMchi-9p0_ctau-10', 'sig_2018_Mchi-94p5_dMchi-9p0_ctau-1', 'sig_2018_Mchi-84p0_dMchi-8p0_ctau-10', 'sig_2018_Mchi-77p0_dMchi-14p0_ctau-100', 'sig_2018_Mchi-84p0_dMchi-8p0_ctau-1', 'sig_2018_Mchi-77p0_dMchi-14p0_ctau-10', 'sig_2018_Mchi-73p5_dMchi-7p0_ctau-100', 'sig_2018_Mchi-77p0_dMchi-14p0_ctau-1', 'sig_2018_Mchi-63p0_dMchi-6p0_ctau-100', 'sig_2018_Mchi-73p5_dMchi-7p0_ctau-10', 'sig_2018_Mchi-73p5_dMchi-7p0_ctau-1', 'sig_2018_Mchi-63p0_dMchi-6p0_ctau-10', 'sig_2018_Mchi-5p5_dMchi-1p0_ctau-100', 'sig_2018_Mchi-5p5_dMchi-1p0_ctau-10', 'sig_2018_Mchi-63p0_dMchi-6p0_ctau-1', 'sig_2018_Mchi-5p5_dMchi-1p0_ctau-1', 'sig_2018_Mchi-5p25_dMchi-0p5_ctau-100', 'sig_2018_Mchi-5p25_dMchi-0p5_ct

In [9]:
#Let us get deep into the cutflow:
#print (sig_histo['cutflow'][sig_samples[0]])
cuts = list(sig_histo['cutflow'][sig_samples[0]].keys())
print (cuts)

['all', 'hasVtx', 'cut4', 'cut5', 'cut6', 'cut7', 'cut8', 'cut9', 'cut10']


Based on the signal file name, i.e. sig_2018_Mchi-55p0_dMchi-10p0_ctau-100, extract the mass point, lifetime, etc and make it into a pandas dataframe: use signalPoint defined in /python_analysis/analysisTools/plotTools.py



In [10]:
#Now we are pointing to the sig_samples:

si = pd.DataFrame.from_dict({s:ptools.signalPoint(s) for s in sig_samples}, orient='index')

print(si)

                                          mchi  dmchi   ctau     m1     m2  \
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100    99.0   18.0  100.0   90.0  108.0   
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100     94.5    9.0  100.0   90.0   99.0   
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10     99.0   18.0   10.0   90.0  108.0   
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1      99.0   18.0    1.0   90.0  108.0   
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100    88.0   16.0  100.0   80.0   96.0   
...                                        ...    ...    ...    ...    ...   
sig_2018_Mchi-110p0_dMchi-20p0_ctau-1    110.0   20.0    1.0  100.0  120.0   
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1       10.5    1.0    1.0   10.0   11.0   
sig_2018_Mchi-105p0_dMchi-10p0_ctau-1    105.0   10.0    1.0  100.0  110.0   
sig_2018_Mchi-105p0_dMchi-10p0_ctau-100  105.0   10.0  100.0  100.0  110.0   
sig_2018_Mchi-105p0_dMchi-10p0_ctau-10   105.0   10.0   10.0  100.0  110.0   

                                         delta  \
sig_2018_Mchi

Access the cut names stored in cutDesc via getCut defined in /python_analysis/analysisTools/plotTools.py. The function removes the redundant junk in all and hasVtx steps.

In [11]:
print (sig_histo ['cutDesc'])   #represents cut names

defaultdict(<class 'str'>, {'all': 'No cuts', 'hasVtx': 'Baseline Selection', 'cut4': 'computing BDT score', 'cut5': 'No b-tagged jets', 'cut6': 'Leading jet |eta| < 2.4', 'cut7': 'Leading jet pT > 80 GeV', 'cut8': 'dPhi(MET,leading jet) > 1.5', 'cut9': 'dPhi(MET,all jets) > 0.75', 'cut10': 'BDT'})


In [12]:
# cutflow: using getCut. No difference. 
cutNames = {cname:ptools.getCut(sig_histo['cutDesc'][cname]) for cname in sig_histo['cutDesc'].keys()}
print(cutNames)

{'all': 'No cuts', 'hasVtx': 'Baseline Selection', 'cut4': 'computing BDT score', 'cut5': 'No b-tagged jets', 'cut6': 'Leading jet |eta| < 2.4', 'cut7': 'Leading jet pT > 80 GeV', 'cut8': 'dPhi(MET,leading jet) > 1.5', 'cut9': 'dPhi(MET,all jets) > 0.75', 'cut10': 'BDT'}


There are four cutflow-related variables in the coffea output:

In [16]:
cf_s = sig_histo['cutflow']              # efficiency
ct_s = sig_histo['cutflow_cts']          # xsec-weighted event count
fr_s = sig_histo['cutflow_vtx_matched'] # fraction that selected vtx is truth-matched, for signal
desc = sig_histo['cutDesc']              #cut names

#---------------------Why cutflow_nevts not included?------------------
raw_s = sig_histo['cutflow_nevts']  
print (raw_s)

{'sig_2018_Mchi-99p0_dMchi-18p0_ctau-100': defaultdict(<class 'int'>, {'all': 11997, 'hasVtx': 4202, 'cut4': 4202, 'cut5': 3601, 'cut6': 3482, 'cut7': 3481, 'cut8': 3481, 'cut9': 3446, 'cut10': 2296}), 'sig_2018_Mchi-94p5_dMchi-9p0_ctau-100': defaultdict(<class 'int'>, {'all': 13659, 'hasVtx': 4363, 'cut4': 4363, 'cut5': 3736, 'cut6': 3655, 'cut7': 3655, 'cut8': 3655, 'cut9': 3611, 'cut10': 2325}), 'sig_2018_Mchi-99p0_dMchi-18p0_ctau-10': defaultdict(<class 'int'>, {'all': 23096, 'hasVtx': 11644, 'cut4': 11644, 'cut5': 9992, 'cut6': 9751, 'cut7': 9749, 'cut8': 9749, 'cut9': 9647, 'cut10': 7739}), 'sig_2018_Mchi-99p0_dMchi-18p0_ctau-1': defaultdict(<class 'int'>, {'all': 28386, 'hasVtx': 10641, 'cut4': 10641, 'cut5': 9126, 'cut6': 8878, 'cut7': 8870, 'cut8': 8870, 'cut9': 8778, 'cut10': 6530}), 'sig_2018_Mchi-88p0_dMchi-16p0_ctau-100': defaultdict(<class 'int'>, {'all': 5352, 'hasVtx': 1851, 'cut4': 1851, 'cut5': 1600, 'cut6': 1556, 'cut7': 1556, 'cut8': 1556, 'cut9': 1538, 'cut10': 104

In [15]:
for process in raw_s.keys():
    print(f'{process} after preselections (raw count) = {raw_s[process]["all"]}')

sig_2018_Mchi-99p0_dMchi-18p0_ctau-100 after preselections (raw count) = 11997
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100 after preselections (raw count) = 13659
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10 after preselections (raw count) = 23096
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1 after preselections (raw count) = 28386
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100 after preselections (raw count) = 5352
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10 after preselections (raw count) = 5548
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100 after preselections (raw count) = 4759
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10 after preselections (raw count) = 31420
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1 after preselections (raw count) = 36408
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10 after preselections (raw count) = 9605
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100 after preselections (raw count) = 4390
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1 after preselections (raw count) = 12329
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10 after preselections (raw count) = 7

In [17]:
cf_s = sig_histo['cutflow']              # efficiency
#print (cf_s)

{'sig_2018_Mchi-99p0_dMchi-18p0_ctau-100': defaultdict(<class 'float'>, {'all': 0.022130613659055123, 'hasVtx': 0.007751382439603958, 'cut4': 0.007751382439603958, 'cut5': 0.006642541578549005, 'cut6': 0.006423078054448954, 'cut7': 0.0064212356942036685, 'cut8': 0.0064212356942036685, 'cut9': 0.0063566162910981185, 'cut10': 0.004235291271042684}), 'sig_2018_Mchi-94p5_dMchi-9p0_ctau-100': defaultdict(<class 'float'>, {'all': 0.018508212315340073, 'hasVtx': 0.005911993485995932, 'cut4': 0.005911993485995932, 'cut5': 0.005062468030165632, 'cut6': 0.004952663031484934, 'cut7': 0.004952663031484934, 'cut8': 0.004952663031484934, 'cut9': 0.004893063542986609, 'cut10': 0.0031505557233270113}), 'sig_2018_Mchi-99p0_dMchi-18p0_ctau-10': defaultdict(<class 'float'>, {'all': 0.045037154783835125, 'hasVtx': 0.02270601642392803, 'cut4': 0.02270601642392803, 'cut5': 0.01948486610572944, 'cut6': 0.019014926056683312, 'cut7': 0.01901102274339839, 'cut8': 0.01901102274339839, 'cut9': 0.0188121793909128,

In [18]:
process = 'sig_2018_Mchi-99p0_dMchi-18p0_ctau-100'
print (process)

sig_2018_Mchi-99p0_dMchi-18p0_ctau-100


In [21]:
print ('Efficiency=  selected events / total events analyzed from ntuples:' ,cf_s[process])

Efficiency=  selected events / total events analyzed from ntuples: defaultdict(<class 'float'>, {'all': 0.022130613659055123, 'hasVtx': 0.007751382439603958, 'cut4': 0.007751382439603958, 'cut5': 0.006642541578549005, 'cut6': 0.006423078054448954, 'cut7': 0.0064212356942036685, 'cut8': 0.0064212356942036685, 'cut9': 0.0063566162910981185, 'cut10': 0.004235291271042684})


In [22]:
print ('Events (xsec weighted):' ,ct_s[process])

Events (xsec weighted): defaultdict(<class 'float'>, {'all': 0.16991947802670063, 'hasVtx': 0.059515333755054696, 'cut4': 0.059515333755054696, 'cut5': 0.0510016222408671, 'cut6': 0.049316575091452756, 'cut7': 0.04930242939734599, 'cut8': 0.04930242939734599, 'cut9': 0.04880627979140864, 'cut10': 0.03251868624854665})


In [23]:
cutNames = {cname:ptools.getCut(sig_histo['cutDesc'][cname]) for cname in sig_histo['cutDesc'].keys()}
print(cutNames)

{'all': 'No cuts', 'hasVtx': 'Baseline Selection', 'cut4': 'computing BDT score', 'cut5': 'No b-tagged jets', 'cut6': 'Leading jet |eta| < 2.4', 'cut7': 'Leading jet pT > 80 GeV', 'cut8': 'dPhi(MET,leading jet) > 1.5', 'cut9': 'dPhi(MET,all jets) > 0.75', 'cut10': 'BDT'}


In [24]:
cut = 'all'

print(process)
print(f'\nEfficiency after {cutNames[cut]}: {cf_s[process][cut]}')

sig_2018_Mchi-99p0_dMchi-18p0_ctau-100

Efficiency after No cuts: 0.022130613659055123


**The efficiency after no cuts applied is less than 1.**

Note that the efficiency at 'No cuts' step is not 1. This is because, 'no cut' is from the point of coffea analyzer, but some cuts already have been applied by the skimmer.

In [26]:
cut = 'hasVtx'

print(process)
print(f'\nEfficiency after {cutNames[cut]}: {cf_s[process][cut]}')

sig_2018_Mchi-99p0_dMchi-18p0_ctau-100

Efficiency after Baseline Selection: 0.007751382439603958
